In [1]:
import nltk, gensim, re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.cluster import KMeans
from scipy import stats

sns.set_context('talk')

### Loading and inspecting the dataset

In [3]:
#Instantiate DataFrame of craigslist data

listings_df = pd.read_csv('data\\CRAIGSLIST.Listings-1.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'data\\CRAIGSLIST.Listings-1.csv'

In [ ]:
# Function to print dataframe columns in a readable format

def format_columns(df):
    print("Columns:")
    for column in df.columns:
        print(column)

In [ ]:
format_columns(listings_df)

In [ ]:
listings_df.RETRIEVED_ON.min()

In [ ]:
listings_df.RETRIEVED_ON.max()

#### The dataset includes listings that were active on the site from 01/18/2020 through 04/26/2020. The 'RETRIEVED_ON' column will no longer be needed for this dataset.

#### Because this tool is intended to be used with a snapshot of current listing data, ['LISTING_MONTH', 'LISTING_DAY, 'LISTING_TIME'] are not relevant.  

#### The location data contained in ['ADDRESS', 'LOCATION', 'CT_ID_10'] are outside the current scope of this project. Despite being particularly important for real estate, the vast majority of goods and services offered on craigslist are filtered by location as much as necessary by user location selection.

In [ ]:
#Drop unnecessary columns from feature set

listings_df.drop(['LISTING_MONTH',
               'LISTING_DAY',
               'LISTING_TIME',
               'RETRIEVED_ON',
               'ADDRESS',
               'LOCATION',
               'CT_ID_10'],
              axis =1,
              inplace = True)

In [ ]:
format_columns(listings_df)

In [ ]:
unique_IDs = listings_df.LISTING_ID.nunique()
row_count = listings_df.shape[0]
excess = row_count - unique_IDs

print('Unique Listing IDs:', unique_IDs)
print('Rows in DataFrame:', row_count)
print('Excess Rows:', excess)
print('Percentage of Dataset that is Excess:', round(excess/row_count, 4) * 100)

#### There are 5,445 observations in the dataset that contain duplicate IDs. Duplicates will be dropped in favor of the more recent listing

In [ ]:
listings_df.drop_duplicates(subset = ['LISTING_ID'], inplace = True, keep = 'last')

In [ ]:
unique_IDs = listings_df.LISTING_ID.nunique()
row_count = listings_df.shape[0]
excess = row_count - unique_IDs

print('Unique Listing IDs:', unique_IDs)
print('Rows in DataFrame:', row_count)
print('Excess Rows:', excess)
print('Percentage of Dataset that is Excess:', round(excess/row_count, 4) * 100)

#### 'LISTING_ID' will now be used as the index of the DataFrame

In [ ]:
listings_df.set_index('LISTING_ID', drop = True, inplace = True)

In [ ]:
listings_df.describe()

In [ ]:
print('Current number of rows in DataFrame:', listings_df.shape[0])

#### The missing data in the pet-related and 'AREA_SQFT' columns need to be accounted for.

In [ ]:
missing_pet_data_count = listings_df.shape[0] - sum(listings_df.ALLOWS_CATS.notnull())
print('Observations with missing pet data:', missing_pet_data_count)
print('Percent of original dataset:', round(missing_pet_data_count/row_count, 4) * 100)

In [ ]:
missing_area_data_count = listings_df.shape[0] - sum(listings_df.AREA_SQFT.notnull())
print('Observations with missing pet data:', missing_area_data_count)
print('Percent of original dataset:', round(missing_area_data_count/row_count, 4) * 100)

#### Because the observations with missing pet-related data compose only 1.95% of the dataset, those observations will be removed.

In [ ]:
#Drop rows without pet data

listings_df = listings_df[listings_df.ALLOWS_CATS.notnull()]

#### However, over half of the observations are missing square footage data, so the missing values will be replaced with the median of the existing values after removing outliers

In [ ]:
with_sqft = listings_df[listings_df.AREA_SQFT.notnull()]
without_sqft = listings_df[listings_df.AREA_SQFT.isnull()]

print('Rows with square footage:', with_sqft.shape[0])
print('Rows without square footage:', without_sqft.shape[0])

In [ ]:
# Check the largest observations for square footage

areas = with_sqft.AREA_SQFT
areas.sort_values(ascending = False).array[:50]

#### According to Wikipedia, the largest house in MA is Castle Hill (https://en.wikipedia.org/wiki/List_of_largest_houses_in_the_United_States) and it has a square footage of 56, 368. The small number of observations that are greater must be erroneous and can be removed from the dataset.

In [ ]:
with_sqft = with_sqft[with_sqft.AREA_SQFT < 56368]

In [ ]:
plt.scatter(with_sqft.AREA_SQFT, with_sqft.PRICE)

In [ ]:
print('Listings with unknown square footage:', sum(with_sqft.AREA_SQFT == 9999))

#### It is apparent that 9999 was used as a value for unknown square footages. There are a number of outliers as well.

In [ ]:
without_sqft = without_sqft.append(with_sqft[with_sqft.AREA_SQFT == 9999])

with_sqft = with_sqft[with_sqft.AREA_SQFT != 9999]

print('Rows with square footage:', with_sqft.shape[0])
print('Rows without square footage:', without_sqft.shape[0])

In [ ]:
z = np.abs(stats.zscore(with_sqft.AREA_SQFT))
outlier_mask = z < 3

with_sqft = with_sqft[outlier_mask]
with_sqft.shape

In [ ]:
plt.scatter(with_sqft.AREA_SQFT, with_sqft.PRICE)

In [ ]:
sum(with_sqft.AREA_SQFT == 0)

In [ ]:
with_sqft = with_sqft[with_sqft.AREA_SQFT != 0]

In [ ]:
with_sqft.shape

In [ ]:
sns.boxplot(x=with_sqft.AREA_SQFT)

In [ ]:
median_sqft = np.median(with_sqft.AREA_SQFT)

In [ ]:
without_sqft.AREA_SQFT = median_sqft

In [ ]:
without_sqft.head()

In [ ]:
listings_df = with_sqft.append(without_sqft)

In [ ]:
listings_df.shape

In [ ]:
listings_df.head()

In [ ]:
listings_df.tail()

In [ ]:
plt.hist(listings_df.PRICE, bins = 25)

In [ ]:
sns.boxplot(x=listings_df.PRICE)

In [ ]:
listings_df.info()

In [ ]:
listings_df.to_csv('\\data\\wrangled_data.csv')

#### The data is now wrangled and ready for engineering!